### Task: Sentiment Classification of Movie Reviews  


Alice is a time traveler who visits different eras in the past to solve important missions. While there, she must always be careful to disguise herself so that no one will know she is from the future. This time, she joined an NLP company in 2014 year and was assigned the task of sentiment analysis on user reviews for movies. Help Alice with this task.

You need to solve sentiment classification task using the imdb movie review dataset. Each review is labeled as either positive (1) or negative (0), indicating its sentiment. You will be provided by basic LinearSVC classifier with TF-IDF features.

You need to solve 3 tasks:

1.   Task1: Text Preprocessing with spaCy (this is your baseline)
2.   Task 2: Adding Part-of-Speech (POS) Features as a TF-IDF for Each POS Category
3.   Task 3: Development of new features to improve classification accuracy

**Note!** Do not change the classifier. Change only cells with TODO mark.



In [ ]:
import os
import random
import re
import numpy as np
import pandas as pd
import spacy

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import (
    TfidfVectorizer,
    CountVectorizer,
)
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
os.environ["PYTHONHASHSEED"] = str(42)

random.seed(42)
np.random.seed(42)

### Loading the dataset

In [ ]:
! gdown --id 1C6TIP8c33fHM6dxs6DoxJeKY6ZXGWpBx
! gdown --id 1K8WBFVVvVlsvIMRG8HiaFkldiyuNkLD2

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1C6TIP8c33fHM6dxs6DoxJeKY6ZXGWpBx
To: /content/imdb_train_hw1.csv
100% 8.25M/8.25M [00:00<00:00, 29.7MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1K8WBFVVvVlsvIMRG8HiaFkldiyuNkLD2
To: /content/imdb_test_hw1.csv
100% 2.10M/2.10M [00:00<00:00, 165MB/s]


In [ ]:
df_train = pd.read_csv("imdb_train_hw1.csv")
df_test = pd.read_csv("imdb_test_hw1.csv")
df_train.sample(5)

,Unnamed: 0,label,text
8681,8681,1,I noticed this movie was getting trashed well ...
2362,2362,1,When it comes to creating a universe George Lu...
6232,6232,0,"""National Treasure"" (2004) is a thoroughly mis..."
1318,1318,1,I must admit - the only reason I bought this m...
543,543,1,Ten out of the 11 short films in this movie ar...


In [ ]:
y_train = df_train["label"]
y_test = df_test["label"]

Since the classes in our dataset are nearly balanced, we can use accuracy as the evaluation metric. Accuracy provides a straightforward measure of how well the model classifies reviews correctly across both sentiment classes.  

However, we will consider the F1-score for a more detailed performance assessment. Even with balanced classes, the model might still be biased towards one class due to feature distributions (e.g., it may predict negative reviews more confidently than positive ones).  

The F1-score, which is the harmonic mean of precision and recall, helps us identify such imbalances. It ensures that both false positives and false negatives are accounted for, providing a better understanding of how well the model performs on each sentiment class.

## 0. LinearSVC with TF-IDF Features  

We will now train a LinearSVC model using TF-IDF (Term Frequency-Inverse Document Frequency) as features.

In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(df_train["text"])
X_test_tfidf = vectorizer.transform(df_test["text"])

In [ ]:
y_train = df_train["label"]
y_test = df_test["label"]

In [ ]:
model = LinearSVC(random_state=42)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)
print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy (TF-IDF): 0.841747984726347
              precision    recall  f1-score   support

           0       0.85      0.84      0.85      1213
           1       0.83      0.84      0.84      1144

    accuracy                           0.84      2357
   macro avg       0.84      0.84      0.84      2357
weighted avg       0.84      0.84      0.84      2357



The model's accuracy using TF-IDF is 0.8417 (84.17%) this our **baseline result**.

## Task1: Text Preprocessing with spaCy

Lemmatize original review texts with [spacy ](https://spacy.io/usage/linguistic-features#lemmatization)library.
With spacy remove:

*   stop words
*   punctuation
*   digits
*   emails
*   numbers
*   empty word

Train classifier with a new tf-idf representation of text. Obtain baseline classification metrics.

In [ ]:
#TODO function take text as an argument and return cleaned text
nlp = spacy.load("en_core_web_sm")


def clean_text(text):
    doc = nlp(text)
    tokens = []
    for token in doc:
        if not (token.is_stop or token.is_punct or token.is_digit or
                token.like_email or token.is_space or token.like_num):
            tokens.append(token.lemma_)

    return ' '.join(tokens)

In [ ]:
df_train["text_lemmatized"] = df_train["text"].apply(clean_text)
df_test["text_lemmatized"] = df_test["text"].apply(clean_text)

In [ ]:
# TODO get tf-idf vectors for your lemmatized texts
X_train_tfidf_lemmatized = vectorizer.fit_transform(df_train["text_lemmatized"])
X_test_tfidf_lemmatized = vectorizer.transform(df_test["text_lemmatized"])

In [ ]:
model = LinearSVC(random_state=42)
model.fit(X_train_tfidf_lemmatized, y_train)
y_pred = model.predict(X_test_tfidf_lemmatized)
print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred))

Accuracy (TF-IDF): 0.8413237165888842


This is your **baseline** metrics!

## Task 2: Adding Part-of-Speech (POS) Features as a TF-IDF for Each POS Category

For each text add part-of-speach (pos) tags as feature in TF-IDF manner. Use Spacy to get pos tag features. Combine them with lemmatized tf-idf features, obtained in the Task1.

For example, if you have two sentences with following tf-idf vectors:

1.   sent1: "The cat sat on the mat." -> [0.63, 0.44, 0.31, 0.31, 0.44, 0, 0]
2.   sent2: "The dog sat on the floor. " -> [0.63, 0, 0.31, 0.31, 0, 0.44, 0.44]

And you obtained the following pos tag features (with dictionary {'det': 1, 'noun': 2, 'verb': 3, 'adp': 0}):

*   sent1: [0.63, 0.63, 0.31, 0.31]
*   sent2: [0.63, 0.63, 0.31, 0.31]


Then final representation should be:

*   sent1: [0.63, 0.44, 0.31, 0.31, 0.44, 0, 0, 0.63, 0.63, 0.31, 0.31]
*   sent2: [0.63, 0, 0.31, 0.31, 0, 0.44, 0.44, 0.63, 0.63, 0.31, 0.31]

**Note!** Do not use pos tags punctuation and empty words

In [ ]:
# TODO function takes text as input and return string with pos tags joined by a space.
def extract_pos_tags(text):
    doc = nlp(text)
    pos_of_tokens = []
    for token in doc:
        if not (token.is_punct or token.is_space):
            pos_of_tokens.append(token.pos_)

    return ' '.join(pos_of_tokens)

In [ ]:
df_train["pos_text"] = df_train["text"].apply(extract_pos_tags)
df_test["pos_text"] = df_test["text"].apply(extract_pos_tags)

We need to bring the features obtained by CountVectorizer for POS tags to the same scale as TF-IDF. The easiest way is to apply TfidfTransformer to the CountVectorizer result.

In [ ]:
#TODO train bag of words with pos tag features, then normalize them with TfidfTransformer, combine with X_train_tfidf_lemmatized
# and X_test_tfidf_lemmatized features, save resulted features to the following variables:
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import hstack


count_vec = CountVectorizer()
pos_count_vector_train = count_vec.fit_transform(df_train["pos_text"])
pos_count_vector_test = count_vec.transform(df_test["pos_text"])

tfidf_transformer = TfidfTransformer()
pos_count_vector_train = tfidf_transformer.fit_transform(pos_count_vector_train)
pos_count_vector_test = tfidf_transformer.transform(pos_count_vector_test)

X_train_combined = hstack([X_train_tfidf_lemmatized, pos_count_vector_train])
X_test_combined = hstack([X_test_tfidf_lemmatized, pos_count_vector_test])

In [ ]:
lr_combined = LinearSVC(random_state=42)
lr_combined.fit(X_train_combined, y_train)
y_pred_combined = lr_combined.predict(X_test_combined)

print("Accuracy (tf-idf + POS):", accuracy_score(y_test, y_pred_combined))

Accuracy (tf-idf + POS): 0.8447178616885872


## Task 3: Development of new features to improve classification accuracy

Come up with another feature or set of features and help Alice improve the quality. Remember that Alice is in the past and does not have access to any . Additional training data cannot be used either. You can use third-party resources to generate features.

Compare with result of your **baseline** from the Task 1. Any improvement will be counted. Use X_train_tfidf_lemmatized and X_test_tfidf_lemmatized, add combine your features with them as in task 2.

In [ ]:
# TODO create your features function here, add feature explanation
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler


def get_custom_feature(text):
    doc = nlp(text)

    tokens = [token.text for token in doc]
    negations = {"not", "no", "never", "n't"}
    features = [
        len(tokens),
        len(text),
        sum(1 for token in doc if token.is_stop),
        sum(1 for token in doc if token.is_punct),
        sum(1 for token in doc if token.is_digit),
        sum(1 for token in doc if token.like_email),
        sum(1 for token in doc if token.is_space),
        sum(1 for token in doc if token.like_num),
        sum(1 for ch in text if ch.isupper()),
        sum(1 for ch in text if ch.islower()),
        len(set(tokens)),
        round(sum(len(token.text) for token in doc) / len(tokens), 4) if tokens else 0,
        sum(1 for ch in text if not ch.isalnum() and not ch.isspace()),
        sum(1 for token in doc if token.pos_ == "ADJ"),
        sum(1 for token in doc if token.pos_ == "ADV"),
        sum(1 for token in doc if token.pos_ == "VERB"),
        sum(1 for token in doc if token.pos_ == "NOUN"),
        sum(1 for token in doc if token.pos_ == "SYM"),
        sum(1 for token in doc if token.lower_ in negations),
        len(doc.ents),
        text.count("!"),
        text.count("?"),
        text.count("$"),
        text.count(":"),
        text.count("."),
        text.count(",")
    ]

    return features


X_train_features = df_train["text"].apply(get_custom_feature)
X_test_features = df_test["text"].apply(get_custom_feature)

X_train_features_df = pd.DataFrame(X_train_features.tolist(), dtype=float)
X_test_features_df = pd.DataFrame(X_test_features.tolist(), dtype=float)

X_train_features_matrix = csr_matrix(X_train_features_df)
X_test_features_matrix = csr_matrix(X_test_features_df)
X_train_combined = hstack([X_train_tfidf_lemmatized, X_train_features_matrix])
X_test_combined = hstack([X_test_tfidf_lemmatized, X_test_features_matrix])

In [ ]:
lr_combined = LinearSVC(random_state=42, dual=False)
lr_combined.fit(X_train_combined, y_train)
y_pred_combined = lr_combined.predict(X_test_combined)

print("Accuracy (tf-idf + Custom feature):", accuracy_score(y_test, y_pred_combined))

Accuracy (tf-idf + Custom feature): 0.8599915146372508
